In [1]:
#####################################
# testing the runner
#################################

import sys
import pickle as pkl
import numpy as np

sys.path.append('/home/edwardsb/repositories/be-SATGOpenFL/openfl/federated/task')

from runner_weightsonly_checkpoint_pytorch import WeightsOnlyPyTorchCheckpointTaskRunner
from nnunet_v1 import train_nnunet


import os
os.environ['nnUNet_raw_data_base']='/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base'
os.environ['nnUNet_preprocessed']='/raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed'
os.environ['RESULTS_FOLDER']='/raid/edwardsb/projects/RANO/NNUnetModels'



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [2]:
def serialize(obj, path):
    with open(path, 'wb') as _file:
        pkl.dump(obj, _file)

In [3]:
def compare_tensordicts(t1, t2):
    results_dict = {}
    for k in t1:
        if k != '__opt_state_needed':
            results_dict[k] = np.linalg.norm(t1[k] - t2[k])

    return np.sum(list(results_dict.values()))

In [4]:
# No longer needed - get the train script to call directly (retraining from scratch as I am starting with a new task and dataset - last samples were repeat samples)
"""

import os
import torch
os.environ['CUDA_VISIBLE_DEVICES']='3'

print(f"GPU enabled: {torch.cuda.is_available()}")

train_nnunet(epochs=1, current_epoch=0, continue_training=False, task='Task543_FakePostOpp_More')
"""


'\n\nimport os\nimport torch\nos.environ[\'CUDA_VISIBLE_DEVICES\']=\'3\'\n\nprint(f"GPU enabled: {torch.cuda.is_available()}")\n\ntrain_nnunet(epochs=1, current_epoch=0, continue_training=False, task=\'Task543_FakePostOpp_More\')\n'

In [4]:
task = 'Task543_FakePostOpp_More'

In [5]:
checkpoint_path = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/' + task + '/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_final_checkpoint.model'

start_path = '/raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/' + task + '/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_START.model'

test_results_folder = '/raid/edwardsb/projects/RANO/NNUnetExperiments/TestOutput/pickled_tensor_dicts'

In [6]:
from runner_pt_utils import DummyDataLoader

dummy_loader = DummyDataLoader(feature_shape=[1, 28, 28], 
                               training_data_size=938, 
                               valid_data_size=10)

In [7]:
runner = WeightsOnlyPyTorchCheckpointTaskRunner(data_loader=dummy_loader, 
                                                checkpoint_in_path=checkpoint_path,
                                                checkpoint_out_path=checkpoint_path, 
                                                checkpoint_init_path=start_path,
                                                device='cuda', 
                                                gpu_num_string='3')

In [8]:
# Initialize to a starting point (replace_checkpoint does so with entire file replacement inluding any model, opt, and metric state)
runner.replace_checkpoint(path_to_replacement=start_path)

# Get model and optimizert state from the starting checkpoint
starting_tensor_dict = runner.get_tensor_dict(with_opt_vars=True)

In [9]:
# This keys value gets popped off every time it is used
starting_tensor_dict['__opt_state_needed'] = True

In [10]:
# when used for training function, allows optimizer state to get reset regardless of the fact that opt_treatment is CONTINUE_LOCAL
testing_with_opt_setting = True

In [12]:
e2_DB_1, _ = runner.train(col_name='BTest', 
                            round_num=0, 
                            input_tensor_dict=starting_tensor_dict, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting)
e2_tdict_1 = runner.get_tensor_dict(with_opt_vars=True)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No

/home/edwardsb/repositories/nnUNet/nnunet/training/network_training/nnUNetTrainer.py:712: RuntimeWarning: invalid value encountered in scalar divide
  global_dc_per_class = [i for i in [2 * i / (2 * i + j + k) for i, j, k in


2024-02-24 11:57:08.411207: lr: 0.009982
2024-02-24 11:57:08.412038: This epoch took 580.366969 s

2024-02-24 11:57:08.485446: saving checkpoint...
2024-02-24 11:57:09.023895: done, saving took 0.61 seconds
BraTS2021_01200 (5, 130, 159, 123)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (4, 130, 160, 123)
patch size: [128 160 112]
steps (x, y, and z): [[0, 2], [0], [0, 11]]
number of tiles: 4
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
no resampling necessary
BraTS2021_01394 (5, 138, 168, 135)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (4, 138, 168, 135)
patch size: [128 160 112]
steps (x, y, and z): [[0, 10], [0, 8], [0, 23]]
number of tiles: 8
using precomputed Gaussian
prediction done
force_separate_z: None interpolation order: 1
no resampling necessary
2024-02-24 11:57:39.281304: finished prediction
2024-02-24 11:57:39.282976: evaluation of raw predictions


/home/edwardsb/repositories/nnUNet/nnunet/evaluation/evaluator.py:381: RuntimeWarning: Mean of empty slice
  all_scores["mean"][label][score] = float(np.nanmean(all_scores["mean"][label][score]))


In [13]:
e2_tdict_1_path = os.path.join(test_results_folder, 'e2_tdict_1.pkl')

serialize(e2_tdict_1, e2_tdict_1_path)

In [14]:
e2_tdict_1['__opt_state_needed'] = True

In [15]:
e3_DB_1, _ = runner.train(col_name='BTest', 
                            round_num=1, 
                            input_tensor_dict=e2_tdict_1, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting)
e3_tdict_1 = runner.get_tensor_dict(with_opt_vars=True)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No

In [16]:
e3_tdict_1_path = os.path.join(test_results_folder, 'e3_tdict_1.pkl')

serialize(e3_tdict_1, e3_tdict_1_path)

In [17]:
starting_tensor_dict['__opt_state_needed'] = True

In [18]:
e2_DB_2, _ = runner.train(col_name='BTest', 
                            round_num=0, 
                            input_tensor_dict=starting_tensor_dict, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting)
e2_tdict_2 = runner.get_tensor_dict(with_opt_vars=True)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No

In [19]:
e2_tdict_2_path = os.path.join(test_results_folder, 'e2_tdict_2.pkl')

serialize(e2_tdict_2, e2_tdict_2_path)

In [28]:
compare_tensordicts(starting_tensor_dict, e2_tdict_1)

52.77177996401065

In [21]:
compare_tensordicts(e2_tdict_1, e2_tdict_2)

36.08117933971383

In [22]:
compare_tensordicts(e2_tdict_1, e3_tdict_1)

36.358491569725885

In [23]:
compare_tensordicts(e2_tdict_2, e3_tdict_1)

45.511301923318314

In [24]:
list(e2_DB_1.items())[:2]

[(TensorKey(tensor_name='train_loss', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(-0.562729, dtype=float32)),
 (TensorKey(tensor_name='val_eval', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(0.69493566))]

In [25]:
list(e3_DB_1.items())[:2]

[(TensorKey(tensor_name='train_loss', origin='BTest', round_number=1, report=True, tags=('metric',)),
  array(-0.61365074, dtype=float32)),
 (TensorKey(tensor_name='val_eval', origin='BTest', round_number=1, report=True, tags=('metric',)),
  array(0.71172945))]

In [26]:
list(e2_DB_2.items())[:2]

[(TensorKey(tensor_name='train_loss', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(-0.5548707, dtype=float32)),
 (TensorKey(tensor_name='val_eval', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(0.70680778))]

In [29]:
starting_tensor_dict['__opt_state_needed'] = True

In [30]:
# validation only test (ah! expecting this to fail though since training alone is not reproducible!)
e2_DB_3, _ = runner.train(col_name='BTest', 
                            round_num=0, 
                            input_tensor_dict=starting_tensor_dict, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting, 
                            validation_only=True)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No

In [31]:
list(e2_DB_3.items())[:2]

[(TensorKey(tensor_name='train_loss', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(-0.5548707, dtype=float32)),
 (TensorKey(tensor_name='val_eval', origin='BTest', round_number=0, report=True, tags=('metric',)),
  array(0.70680778))]

In [ ]:
# Now train again to try and change the state, then I'll validate again (having moved the output json file)


In [11]:
# This one run after resetting the notebook and only running the import etc cells and this cell
e2_DB_4, _ = runner.train(col_name='BTest', 
                            round_num=0, 
                            input_tensor_dict=starting_tensor_dict, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting, 
                            validation_only=False)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No

/home/edwardsb/repositories/nnUNet/nnunet/training/network_training/nnUNetTrainer.py:712: RuntimeWarning: invalid value encountered in scalar divide
  global_dc_per_class = [i for i in [2 * i / (2 * i + j + k) for i, j, k in


2024-02-24 13:27:44.226924: lr: 0.009982
2024-02-24 13:27:44.227646: This epoch took 583.183552 s

2024-02-24 13:27:44.298261: saving checkpoint...
2024-02-24 13:27:44.833953: done, saving took 0.61 seconds
BraTS2021_01200 (5, 130, 159, 123)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (4, 130, 160, 123)
patch size: [128 160 112]
steps (x, y, and z): [[0, 2], [0], [0, 11]]
number of tiles: 4
computing Gaussian
done
prediction done
force_separate_z: None interpolation order: 1
no resampling necessary
BraTS2021_01394 (5, 138, 168, 135)
debug: mirroring True mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: True
data shape: (4, 138, 168, 135)
patch size: [128 160 112]
steps (x, y, and z): [[0, 10], [0, 8], [0, 23]]
number of tiles: 8
using precomputed Gaussian
prediction done
force_separate_z: None interpolation order: 1
no resampling necessary
2024-02-24 13:28:14.786019: finished prediction
2024-02-24 13:28:14.787740: evaluation of raw predictions


/home/edwardsb/repositories/nnUNet/nnunet/evaluation/evaluator.py:381: RuntimeWarning: Mean of empty slice
  all_scores["mean"][label][score] = float(np.nanmean(all_scores["mean"][label][score]))


In [13]:
starting_tensor_dict['__opt_state_needed'] = True

In [14]:
# validation only test (ah! expecting this to fail though since training alone is not reproducible!)
e2_DB_5, _ = runner.train(col_name='BTest', 
                            round_num=0, 
                            input_tensor_dict=starting_tensor_dict, 
                            epochs=1, 
                            testing_with_opt_setting=testing_with_opt_setting, 
                            validation_only=True)

Brandon DEBUG - task: Task543_FakePostOpp_More, preprocessing_output_dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed

Brandon in get_default_configuration


Brandon - DEBUG - dataset dir: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More and plans file: /raid/edwardsb/projects/RANO/BraTS22_pretending_tobe_postopp/nnUNet_raw_data_base/nnUNet_preprocessed/Task543_FakePostOpp_More/nnUNetPlansv2.1_plans_3D.pkl

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '0000', 1: '0001', 2: '0002', 3: '0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class No